# Laboratorio 1: Detección de Phishing

## Objetivos

- Realizar una revisión de la literatura para identificar características potenciales en las URLs de phishing.
- Implementar un modelo de *Machine Learning* para clasificar si un dominio es legítimo o phishing.

## Integrantes

- **Javier Prado** — 21486  
- **Bryan España** — 21550

# Parte 1 - Ingenieria de caracteristicas
## Exploracion de datos
1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset_pishing.csv')

In [3]:
df.head()

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como
“pishing”. ¿Está balanceado el dataset?


In [4]:
df.status.value_counts()

status
legitimate    5715
phishing      5715
Name: count, dtype: int64

In [ ]:
df['status'] = df['status'].map({'legitimate': 0, 'phishing': 1})
df.head()

El dataset tiene la misma cantidad de url tanto como con estado 'legitimate' y 'pishing' por lo cual este dataset esta balanceado

## Derivación de características
Revise los artículos proporcionados, especialmente en el análisis de las URLs. En base a su análisis
responda las siguientes preguntas:
1. ¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo
de vida del dominio, o las características de la página Web?
- El análisis de URLs presenta varias ventajas significativas frente a otros enfoques utilizados en la detección de phishing, como el análisis del tiempo de vida del dominio o las características del contenido de la página web. Una ventaja es que el análisis de URLs es especialmente eficaz frente a ataques de tipo zero-day, ya que no depende de listas negras ni de información histórica del dominio. Por ejemplo, el tiempo de vida del dominio, aunque es una característica útil, suele requerir acceso a servicios externos como WHOIS, lo que introduce retrasos y dependencia de terceros. Varios estudios destacan que los atacantes suelen ocultar o minimizar el contenido de las páginas phishing, utilizando sitios con muy poca información o contenido dinámico, lo que dificulta los enfoques basados en análisis de páginas web. En estos casos, la URL sigue siendo una fuente de información confiable y accesible para la detección del ataque

2. ¿Qué características de una URL son más prometedoras para la detección de phishing?
Algunas caracteristicas de las URLs para detectar pishing es que la estructura de la URL son especialmente relevantes, dado que se busca la longitud total de la URL, el número de subdominios, la cantidad de '.' y el uso de direcciones IP en lugar de nombres de dominio. Por lo que las URL de pishing suelen ser más largas y complejas con el objetivo de confundir al user. Tambien otra caracteristica es presencias como simbolos @, -, _, // y otros sibolos no alfanumericos son común en URL maliciosas. Tanvuen aun que no se considera estrictamente parte de la URL, las características temporales asociadas al dominio, como la reciente creación del dominio o cambios frecuentes en la configuración, complementan eficazmente el análisis cuando están disponibles.

En base a la respuesta anterior escriba al menos quince funciones basadas en los artículos, para
derivar características que un modelo pueda utilizar y añada dichas características al dataset original.
Incluya dentro de las quince funciones, la entropía de Shanon y relativa.

In [1]:
from urllib.parse import urlparse
from collections import Counter
import math

In [ ]:
# Longitud de la URL
def url_length(url):
    return len(url)

In [ ]:
# Longitud del dominio
def domain_length(url):
    return len(urlparse(url).netloc)

In [ ]:
# Número de subdominios
def count_subdomains(url):
    domain = urlparse(url).netloc
    return domain.count('.')

In [ ]:
# Uso de direccion ip en lugar de dominio
def has_ip_address(url):
    domain = urlparse(url).netloc
    return int(any(char.isdigit() for char in domain))

In [ ]:
# Cantidad de digitos en la URL
def count_digits(url):
    return sum(char.isdigit() for char in url)  

In [ ]:
# Cantidad de letras
def count_letters(url):
    return sum(char.isalpha() for char in url)

In [ ]:
# cantidad de caracteres especiales
def count_special_characters(url):
    return sum(not char.isalnum() for char in url)

In [ ]:
# presencia del simbolo @
def has_at_symbol(url):
    return int('@' in url)

In [ ]:
# Presencua de guioes
def has_hyphen(url):
    return int('-' in url)

In [ ]:
# Presencia de doble slash //
def has_double_slash(url):
    return int(url.find('//', url.find('//') + 2) != -1)

In [ ]:
# Uso de HTTPS
def uses_https(url):
    return int(urlparse(url).scheme == 'https')

In [ ]:
# numero de parametros en la URL
def count_parameters(url):
    return urlparse(url).query.count('&') + 1 if urlparse(url).query else 0

In [ ]:
# shannon entropy H(X)=−∑p(x)log2​p(x)) para cada caracter en la URL
def shannon_entropy(url):
    if not url:
        return 0.0

    char_counts = Counter(url)
    total_chars = len(url)

    entropy = 0.0
    for count in char_counts.values():
        p = count / total_chars
        entropy -= p * math.log2(p)

    return entropy

In [ ]:
# relative entropy 
def relative_entropy(url):
    if not url:
        return 0.0

    unique_chars = len(set(url))
    if unique_chars <= 1:
        return 0.0

    max_entropy = math.log2(unique_chars)
    return shannon_entropy(url) / max_entropy